In [1]:
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))

print(codes)

[36, 162, 163, 165, 8364, 164]


In [ ]:
codes = [ord(symbol) for symbol in symbols]

In [32]:
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [33]:
deck = FrenchDeck()
deck[12::13]


[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

In [2]:
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *(f'{x:.3f}' for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.009 0.008 0.008 0.008 0.008
listcomp + func : 0.012 0.012 0.011 0.011 0.011
filter + lambda : 0.009 0.009 0.009 0.009 0.009
filter + func   : 0.008 0.008 0.008 0.008 0.008


In [3]:
colors = ['black','white']
sizes = ['S','M','L']
tshirts = [(color,size) for color in colors for size in sizes]
print(tshirts)

[('black', 'S'), ('black', 'M'), ('black', 'L'), ('white', 'S'), ('white', 'M'), ('white', 'L')]


In [5]:
*head, a, b = range(3)
print(head)


[0]


In [6]:
*head, a, b = range(1)
print(head)

ValueError: not enough values to unpack (expected at least 2, got 1)

In [11]:
invoice = """
... 1909  Pimoroni PiBrella                 $17.50 3 $52.50
... 1489  6mm Tactile Switch x20            $4.95  2 $9.90
... 1510  Panavise Jr. - PV-201             $28.00 1 $28.00
... 1601  PiTFT Mini Kit 320x240            $34.95 1 $34.95
... """

SKU = slice(0,6)
DESCRIPTION = slice(6,40)
line_itimes = invoice.split('\n')[:]
for item in line_itimes:
    print(item[SKU],item[DESCRIPTION])

 
1909   Pimoroni PiBrella                 
1489   6mm Tactile Switch x20            
1510   Panavise Jr. - PV-201             
1601   PiTFT Mini Kit 320x240            
 


In [35]:
my_dict = {
    'a':1,
    'b':2
}

In [40]:
print(my_dict.setdefault('d',None))
print(my_dict)

None
{'a': 1, 'b': 2, 'c': [3, 3], 'd': None}


In [12]:
import typing

class DemoNTClass(typing.NamedTuple):
    a: int
    b: float
    c = 'spam'

In [13]:
DemoNTClass.__annotations__

{'a': int, 'b': float}

In [14]:
from dataclasses import dataclass

@dataclass
class DemoDataClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [17]:
DemoDataClass.__annotations__

{'a': int, 'b': float}

In [18]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)

In [43]:
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum, auto
from datetime import date

class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)

    def __repr__(self) -> str:
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' '*4
        res = [f'{cls_name}(']
        for name in vars(self):
            value = getattr(self, name)
            res.append(f'{indent}{name} = {value!r},')
        
        res.append(')')
        return '\n'.join(res)

description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition', ['Martin Fowler','Kent Beck'], date(2018,11,19), ResourceType.BOOK, description, 'EN', ['computer programming', 'OOP'])

In [44]:
book

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

In [42]:
class Averager:
    count = 0
    total = 0


    def __call__(self, number):
        self.total += number
        self.count += 1
        return self.total/self.count

avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [44]:
from array import array
import reprlib
import math
from typing import Any
import functools
import operator

class Vector:

    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+bytes(self._components))
    
    def __eq__(self, other) -> bool:
        return len(self) == len(other) and all(a == b for a, b in zip(self,other))
    
    def __hash__(self) -> int:
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.hypot(*self)
        
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]
    
    __match_args__ = ('x','y','z','t')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
    
    def __setattr__(self, name, value) -> None:
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = 'readonly attribute {name!r}'
            elif name.islower():
                error = 'cannot set attributes "a" to "z" in {cls.__name__}'
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
    


In [50]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level+1)
    
def display(cls):
    for cls_name, level in tree(cls):
        indent = ' '*4*level
        print(f'{indent}{cls_name}')
    
display(BaseException)

BaseException
    Exception
        TypeError
            FloatOperation
            MultipartConversionError
        StopAsyncIteration
        StopIteration
        ImportError
            ModuleNotFoundError
                PackageNotFoundError
            ZipImportError
        OSError
            ConnectionError
                BrokenPipeError
                ConnectionAbortedError
                ConnectionRefusedError
                ConnectionResetError
                    RemoteDisconnected
            BlockingIOError
            ChildProcessError
            FileExistsError
            FileNotFoundError
            IsADirectoryError
            NotADirectoryError
            InterruptedError
                InterruptedSystemCall
            PermissionError
            ProcessLookupError
            TimeoutError
            UnsupportedOperation
            herror
            gaierror
            SSLError
                SSLCertVerificationError
                SSLZeroReturnErr

In [ ]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints

class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
    
    def __set__(self, instance, value):
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

class Checked:
    @classmethod
    def _fields(cls):
        return get_type_hints(cls)
    
    def __init_subclass__(subclass) -> None:
        super().__init_subclass__()
        for name, constructor in subclass._fields().items():
            setattr(subclass, name, Field(name, constructor))
    
    def __init__(self, **kwargs):
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unkown_attrs(*kwargs)
    
    def __setattr__(self, name: str, value: Any) -> None:
        if name in self._fields():
            cls = self.__class__
            descriptor = getattr(cls, name)
            descriptor.__set__(self, name)
        else:
            self.__flag_unkown_attrs(name)
    
    def __flag_unkown_attrs(self, *names):
        plural = 's' if len(names)>1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')
    
    def _asdict(self):
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items() if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(f'{key}={value!r}' for key, value in self._asdict().items())
        return f'{self.__class__.__name__}({kwargs})'
    


In [2]:
import missions
import reader.model
print(reader.model.supported_cotrans)
print(reader.model.supported_missions)

[<function rbsp_uvw2gse at 0x000001213E5891B0>, <function gse2rbsp_uvw at 0x000001213E588E50>, <function gse2rbsp_mgse at 0x000001213E5FB010>, <function rbsp_mgse2gse at 0x000001213E5FB490>, <function themis_dsl2gse at 0x000001213E5FB6D0>, <function gse2themis_dsl at 0x000001213E5FB5B0>]
{'rbsp': <class 'missions.rbsp.RBSP'>, 'themis': <class 'missions.themis.THEMIS'>}


In [3]:
from missions.rbsp.rbsp import RBSP

print(RBSP.name)
print(RBSP.efw.read_efield)

rbsp
<bound method EFW.read_efield of <missions.rbsp.efw.EFW object at 0x00000293C8016E90>>


In [7]:
class Quantity:

    def __set_name__(self, owner, name):
        self.storage_name = name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    

class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price) -> None:
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight*self.price

truffle = LineItem('White trffle', 100, 1)
print(truffle)

In [22]:
class Mission:
    probes = None

    def __init__(self, probe=None):
        if self.probes is None:
            self.probe = None
        else:
            if probe in self.probes:
                self.probe = probe
            else:
                raise ValueError(f"Invalid probe {probe} for {self.__class__.__name__}. Must be one of {self.probes}")

class RBSP(Mission):
    probes = ['a','b']

class THEMIS(Mission):
    probes = ['a','b','c','d','e']

class OMNI(Mission):
    probes = None


rbspa = RBSP('a')
print(rbspa.probes)
print(rbspa.probe)

themis_d = THEMIS('d')
print(themis_d.probe)

omni = OMNI()
print(omni.probes)
print(omni.probe)

rbsp_c = RBSP('c')
print(rbsp_c.probe)

['a', 'b']
a
d
None
None


ValueError: Invalid probe c for RBSP. Must be one of ['a', 'b']